#Рекурентные сети для обработки последовательностей

Обучить GRU, LSTM для предсказания временного ряда на примере https://www.kaggle.com/c/favorita-grocery-sales-forecasting (для каждого типа продуктов)

In [1]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

# Define data root directory
#data_dir = "./data/"

In [2]:
# Монтируем ГуглДиск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!p7zip -d '/content/drive/MyDrive/Нейронные сети/Фреймворк_PyTorch/items.csv.7z'


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive . ети/Фреймворк_PyTorch/                                                                       1 file, 14315 bytes (14 KiB)

Extracting archive: /content/drive/MyDrive/Нейронные сети/Фреймворк_PyTorch/items.csv.7z
--
Path = /content/drive/MyDrive/Нейронные сети/Фреймворк_PyTorch/items.csv.7z
Type = 7z
Physical Size = 14315
Headers Size = 122
Method = LZMA2:17
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       101841
Compressed: 14315


In [2]:
train = pd.read_csv('items.csv')

In [3]:
train.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [4]:
train = train.drop('perishable', axis=1)

In [10]:
!p7zip -d '/content/drive/MyDrive/Нейронные сети/Фреймворк_PyTorch/test.csv.7z'


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive . ети/Фреймворк_PyTorch/                                                                       1 file, 4885065 bytes (4771 KiB)

Extracting archive: /content/drive/MyDrive/Нейронные сети/Фреймворк_PyTorch/test.csv.7z
--
Path = /content/drive/MyDrive/Нейронные сети/Фреймворк_PyTorch/test.csv.7z
Type = 7z
Physical Size = 4885065
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

  0%     26% - test.csv                53% - test.csv                79% - test.csv                

In [5]:
test = pd.read_csv('test.csv')

In [6]:
test.head()

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


In [7]:
train_df = test.drop('onpromotion', axis=1)

In [8]:
train_df = train_df.merge(train, on='item_nbr')

In [9]:
train_df.head()

,id,date,store_nbr,item_nbr,family,class
0,125497040,2017-08-16,1,96995,GROCERY I,1093
1,125500941,2017-08-16,2,96995,GROCERY I,1093
2,125504842,2017-08-16,3,96995,GROCERY I,1093
3,125508743,2017-08-16,4,96995,GROCERY I,1093
4,125512644,2017-08-16,5,96995,GROCERY I,1093


In [10]:
train_df = train_df.drop(['family', 'id', 'store_nbr', 'item_nbr'], axis=1)

In [11]:
# The scaler objects will be stored in this dictionary so that our output test data from the model can be re-scaled during evaluation
label_scalers = {}

train_x = []
train_y = []
test_x = {}
test_y = {}

#for file in tqdm_notebook(os.listdir(data_dir)): 
 #   # Skipping the files we're not using
 #   if file[-4:] != ".csv" or file == "pjm_hourly_est.csv":
 #       continue
    
 #   # Store csv file in a Pandas DataFrame
 #   df = pd.read_csv('{}/{}'.format(data_dir, file), parse_dates=[0])
    # Processing the time data into suitable input formats
 #   df['hour'] = df.apply(lambda x: x['Datetime'].hour,axis=1)
train_df['date'] = pd.to_datetime(train_df['date'])
train_df['dayofweek'] = train_df.apply(lambda x: x['date'].dayofweek,axis=1)
train_df['month'] = train_df.apply(lambda x: x['date'].month,axis=1)
train_df['dayofyear'] = train_df.apply(lambda x: x['date'].dayofyear,axis=1)
train_df = train_df.sort_values("date").drop("date",axis=1)
    
# Scaling the input data
sc = MinMaxScaler()
label_sc = MinMaxScaler()
data = sc.fit_transform(train_df.values)
# Obtaining the Scale for the labels(usage data) so that output can be re-scaled to actual value during evaluation
label_sc.fit(train_df.iloc[:,0].values.reshape(-1,1))
label_scalers['file_sc'] = label_sc
    
# Define lookback period and split inputs/labels
lookback = 90
inputs = np.zeros((len(data)-lookback,lookback,train_df.shape[1]))
labels = np.zeros(len(data)-lookback)
    
for i in range(lookback, len(data)):
    inputs[i-lookback] = data[i-lookback:i]
    labels[i-lookback] = data[i,0]
inputs = inputs.reshape(-1,lookback,train_df.shape[1])
labels = labels.reshape(-1,1)
    
# Split data into train/test portions and combining all data from different files into a single array
test_portion = int(0.1*len(inputs))
if len(train_x) == 0:
    train_x = inputs[:-test_portion]
    train_y = labels[:-test_portion]
else:
    train_x = np.concatenate((train_x,inputs[:-test_portion]))
    train_y = np.concatenate((train_y,labels[:-test_portion]))
test_x['file'] = (inputs[-test_portion:])
test_y['file'] = (labels[-test_portion:])

In [12]:
batch_size = 1024
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [13]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()#.to(device)
        return hidden

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),#.to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())#.to(device))
        return hidden

In [14]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
        
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.float(), h)
            loss = criterion(out, label.float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.clock()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.clock()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [ ]:
lr = 0.001
gru_model = train(train_loader, lr, model_type="GRU")
Lstm_model = train(train_loader, lr, model_type="LSTM")

Starting Training of GRU model


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [ ]:
gru_outputs, targets, gru_sMAPE = evaluate(gru_model, test_x, test_y, label_scalers)

In [ ]:
lstm_outputs, targets, lstm_sMAPE = evaluate(Lstm_model, test_x, test_y, label_scalers)